<hr style="height:2px;">

# Demo: Application of trained neural network for isotropic reconstruction of ...

### Notes 

- Assumes that training was already completed via [training.ipynb](training.ipynb).
- The trained CARE network is here applied to the same image that the model was trained on (data generated via [datagen.ipynb](datagen.ipynb)).  
Of course, in practice one would typically use it to restore images that the model hasn't seen during training.
- Documentation available: http://csbdeep.bioimagecomputing.com/doc/

<hr style="height:2px;">
# Setup 

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
from tifffile import imread
from csbdeep.models import CARE
from csbdeep.predict import PercentileNormalizer, PadAndCropResizer
from csbdeep.plot_utils import plot_some

In [ ]:
#try:
#    import gputools
#    from csbdeep.tf import limit_gpu_memory
#    limit_gpu_memory(fraction=3/4)    
#except ImportError:
#    pass

Expect data like this:

    raw_data
    ├── test_stacks_sub_4
    │   └── stack_low_sub_4_03.tif
    └── training_stacks
        ├── high
        │   ├── stack_00.tif
        │   ├── stack_01.tif
        │   └── stack_02.tif
        └── low
            ├── stack_00.tif
            ├── stack_01.tif
            └── stack_02.tif

<hr style="height:2px;">

# Raw 3D image stack with low z resolution

In [ ]:
x = imread('raw_data/test_stacks_sub_4/stack_low_sub_4_03.tif')
axes = 'ZYX'
subsample = 4
print('image size       =', x.shape)
print('image axes       =', axes)
print('subsample factor =', subsample)

plt.figure(figsize=(15,10))
plt.imshow(x[:,:,0],aspect=subsample)
plt.axis('off')
None;

<hr style="height:2px;">

# Model

Load trained model (located in folder `my_model`) from disk.  
The configuration was saved during training and is automatically loaded when `CARE` is initialized with `config=None`.

In [ ]:
model = CARE(config=None, name='my_model')
model.load_weights()

- Select appropriate normalization
- Choose how to resize the image to be able to apply the model

In [ ]:
normalizer = PercentileNormalizer(1,99.8)

## Apply CARE network to raw image

Predict the restored image

In [ ]:
from scipy.ndimage.interpolation import zoom
from csbdeep.utils import axes_dict
factors = np.ones(x.ndim)
factors[axes_dict(axes)['Z']] = subsample
x_upscaled = zoom(x,factors,order=1)
x_upscaled.shape

In [ ]:
%%time

restored = model.predict(x_upscaled, axes, normalizer, n_tiles=8)

print('input  (%s) = %s' % (axes, str(x_upscaled.shape)))
print('output (%s) = %s' % (axes, str(restored.shape)))
print()

In [ ]:
from csbdeep.utils import save_tiff_imagej_compatible
save_tiff_imagej_compatible('restored_stack_low_sub_4_03.tif',restored,axes)

<hr style="height:2px;">

# Reconstructed image via CARE network

In [ ]:
plt.figure(figsize=(15,10))
plt.imshow(x_upscaled[:,:,0])
plt.axis('off')

plt.figure(figsize=(15,10))
plt.imshow(restored[:,:,0])
plt.axis('off')
None;